In [3]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

pdf_text = extract_text_from_pdf("pdfs/2201.08706.pdf")

In [5]:
pdf_text

'1\nSparseAlign: A Super-Resolution Algorithm for\nAutomatic Marker Localization and Deformation\nEstimation in Cryo-Electron Tomography\nPoulami Somanya Ganguly1,2, Felix Lucka1, Holger Kohr3, Erik Franken3, Hermen Jan Hupkes2, and K. Joost\nBatenburg1,4\n1Computational Imaging, Centrum Wiskunde & Informatica, Amsterdam, The Netherlands\n2Mathematical Institute, Leiden University, Leiden, The Netherlands\n3Thermo Fisher Scientific, Eindhoven, The Netherlands\n4Leiden Institute of Advanced Computer Science, Leiden University, Leiden, The Netherlands\nAbstract—Tilt-series alignment is crucial to obtaining high- and the resulting transmitted beam lands on the camera to\nresolution reconstructions in cryo-electron tomography. Beam- form a TEM image. We show a schematic of TEM image\ninducedlocaldeformationofthesampleishardtoestimatefrom formation in Fig. 1(a). For biological samples, the observed\nthe low-contrast sample alone, and often requires fiducial gold\nimage contrast is mainly ph

In [4]:
from openai import OpenAI
from dotenv import load_dotenv
import os
load_dotenv()

client = OpenAI()
client.api_key = os.getenv("OPENAI_API_KEY")
completion = client.chat.completions.create(
    model="gpt-4o",
    store=True,
    messages = [
    {
        "role": "system",
        "content": (
            "You are a data extraction assistant. The user will provide the text from a PDF. "
            "Identify and list all email addresses that appear in the text (i.e., strings containing '@'). "
            "If you find no email addresses, return 'None'. "
            "Output only the emails or 'None'—no additional explanations."
        )
    },
    {
        "role": "user",
        "content": pdf_text
    }
    ],
    temperature=0.2,
)

# Extract and print the message content
message_content = completion.choices[0].message.content
print(message_content)

None
